In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [ ]:
TEIS = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\Python\Projects\ddid-rasberry-pi\data\TEIS Exit Data for NSS.xlsx", sheet_name = 3)

In [ ]:
TEIS_subset = TEIS[['Child ID', 'DOB', 'Initial IFSP Date', 'Exit Reason', 'Exit Date', 'Days from I-IFSP to Exit', 'Exit Note']]
TEIS_subset

## Recalculate the Days of Service for Each Child by Considering the Notes

### Removing Rows with Null Values in the Exit Note Column

In [ ]:
TEIS_filt = TEIS_subset[TEIS_subset['Exit Note'].notnull()]
pd.set_option('display.max_colwidth', None)
TEIS_filt

### Filtering for Exit Notes that contain Dates

In [ ]:
TEIS_with_dates = TEIS_filt.loc[TEIS_filt['Exit Note'].str.contains(r"(\d+/\d+/\d+)", case=False)]
TEIS_with_dates

### Filtering for Exit Notes that contain the phrase 'Reopened'

In [ ]:
TEIS_reopened = TEIS_with_dates.loc[TEIS_with_dates['Exit Note'].str.contains('reopened[a-z]*', flags=re.I, regex=True)]
TEIS_reopened

## Testing Regex

In [ ]:
z = TEIS_reopened.loc[ 5, 'Exit Note']
z

In [ ]:
#isolating the dates
date = re.findall(r'\d{2}/\d{2}/\d{4}', z)

for dates in date :
    print(dates)

In [ ]:
#isolating the string/date column to confirm I'm pulling the desired dates
string = re.findall(r'Reopened: \d{2}/\d{2}/\d{4} Exit Date: \d{2}/\d{2}/\d{4}', z)
string

In [ ]:
y = TEIS_reopened.loc[30, 'Exit Note']
y

In [ ]:
#ensuring that this expression will extract every occurance of this pattern within the string
re.findall(r'Reopened: \d{2}/\d{2}/\d{4} Exit Date: \d{2}/\d{2}/\d{4}', y)

In [ ]:
#splitting the pattern by Reopened/Exit Date so that they can go in separate columns
w = re.findall(r'Reopened: \d{2}/\d{2}/\d{4}', y)
v = re.findall(r'Exit Date: \d{2}/\d{2}/\d{4}', y)

In [ ]:
ww = " ".join(map(str,w))
type(ww)

In [ ]:
#extracting just the date from the reopened/exit date string
re.findall(r'\d{2}/\d{2}/\d{4}', ww)

In [ ]:
#try creating a for loop to extract the dates if converting to a string doesn't work
re.findall(r'\d{2}/\d{2}/\d{4}', w[0])

## Using Regex on DataFrame

In [ ]:
x = TEIS_reopened[['Exit Note']]
x

In [ ]:
dates = TEIS_reopened['Exit Note'].str.findall(r'\d{2}/\d{2}/\d{4}')
dates

In [ ]:
string_col = TEIS_reopened['Exit Note'].str.extract(r'(Reopened: \d{2}/\d{2}/\d{4} Exit Date: \d{2}/\d{2}/\d{4})')
string_col

In [ ]:
reopened_dates = TEIS_reopened['Exit Note'].str.extract(r'(Reopened: \d{2}/\d{2}/\d{4})')
reopened_dates           

In [ ]:
exit_dates = TEIS_reopened['Exit Note'].str.extract(r'(Exit Date: \d{2}/\d{2}/\d{4})')
exit_dates

**Creating New Column for Reopened Dates** 

In [ ]:
TEIS_reopened['Reopened Dates'] = TEIS_reopened['Exit Note'].str.extract(r'Reopened: (\d{2}/\d{2}/\d{4})')

**Creating New Column for Exit Dates** 

In [ ]:
TEIS_reopened['Exit Dates'] = TEIS_reopened['Exit Note'].str.extract(r'Exit Date: (\d{2}/\d{2}/\d{4})')

In [ ]:
TEIS_reopened

**Converting to Datetime**

In [ ]:
TEIS_reopened.dtypes

In [ ]:
TEIS_reopened['Reopened Dates'] = pd.to_datetime(TEIS_reopened['Reopened Dates'])

In [ ]:
TEIS_reopened.dtypes

In [ ]:
TEIS_reopened['Exit Dates'] = pd.to_datetime(TEIS_reopened['Exit Dates'])

In [ ]:
TEIS_reopened.dtypes

**Subtracting the Dates**

In [ ]:
for index, row in TEIS_reopened.iterrows():
    if ((row['Reopened Dates'] > row['Initial IFSP Date']) and
        (row['Reopened Dates'] < row['Exit Date']) and
        (row['Exit Dates'] > row['Initial IFSP Date']) and
        (row['Exit Dates'] < row['Exit Date'])):
        time_elapsed = (row['Reopened Dates'] - row['Exit Dates']).days
        TEIS_reopened.at[index, 'time_elapsed'] = time_elapsed
    
    else:
        TEIS_reopened.at[index, 'time_elapsed'] = np.nan   

In [ ]:
TEIS_reopened.head()

In [ ]:
TEIS_reopened.dtypes

**Find How Many Times Reopened/Exit Date is Mentioned**

In [ ]:
len(re.findall(r'Reopened: \d{2}/\d{2}/\d{4}', y))

In [ ]:
TEIS_reopened['Number of Reopened Dates'] = TEIS_reopened['Exit Note'].str.count(r'(Reopened: \d{2}/\d{2}/\d{4})')

In [ ]:
TEIS_reopened['Number of Exit Dates'] = TEIS_reopened['Exit Note'].str.count(r'Exit Date: \d{2}/\d{2}/\d{4}')

In [ ]:
TEIS_reopened.shape

**Subset the rows that have a count = 1**

In [ ]:
TEIS_single_date = TEIS_reopened[(TEIS_reopened['Number of Reopened Dates'] == 1) & (TEIS_reopened['Number of Exit Dates'] == 1)]
TEIS_single_date

In [ ]:
TEIS_single_date['Real IFSP to Exit Time (Days)'] = (TEIS_single_date['Days from I-IFSP to Exit'] - TEIS_single_date['time_elapsed'])
TEIS_single_date.shape

**Subset the rows with Reopened = 1 and Exit Date > 1**

In [ ]:
TEIS_diff_dates = TEIS_reopened[(TEIS_reopened['Number of Reopened Dates'] <= 1) & (TEIS_reopened['Number of Exit Dates'] > 1)]
TEIS_diff_dates.head()

**Subset the rows that have a count > 1**

In [ ]:
TEIS_multiple_dates = TEIS_reopened[(TEIS_reopened['Number of Reopened Dates'] > 1) & (TEIS_reopened['Number of Exit Dates'] > 1)]
TEIS_multiple_dates.head(2)

**Create subset to analyze rows w/ count > 1**

In [ ]:
child_info = TEIS_multiple_dates[['Child ID','Initial IFSP Date', 'Exit Date']]

In [ ]:
multi_reopen_dates = TEIS_multiple_dates['Exit Note'].str.findall(r'Reopened: (\d{2}/\d{2}/\d{4})')
multi_reopen_dates

In [ ]:
multi_exit_dates = TEIS_multiple_dates['Exit Note'].str.findall(r'Exit Date: (\d{2}/\d{2}/\d{4})')
multi_exit_dates

In [ ]:
subset = pd.concat([child_info, multi_reopen_dates, multi_exit_dates], axis = 1).reset_index(drop = True)
subset.columns = ['Child ID', 'Initial IFSP Date', 'Exit Date', 'Reopen Dates', 'Exit Dates']
subset

In [ ]:
subset_expand_reopen = pd.DataFrame(subset['Reopen Dates'].tolist(), columns = ['Reopen 0', 'Reopen 1', 'Reopen 2', 'Reopen 3', 'Reopen 4', 'Reopen 5']).apply(pd.to_datetime)

In [ ]:
subset_expand_exit = pd.DataFrame(subset['Exit Dates'].tolist(), columns = ['Exit 0', 'Exit 1', 'Exit 2', 'Exit 3', 'Exit 4', 'Exit 5']).apply(pd.to_datetime)

In [ ]:
subset_expanded = pd.concat([subset, subset_expand_reopen, subset_expand_exit], axis = 1)
subset_expanded

In [ ]:
subset_expanded.info()

**Subtracting Across Columns**

In [ ]:
for index, row in subset_expanded.iterrows():
    new_diff = 0
    for num in range(0,6):
        if ((row[f'Exit {num}'] != row[f'Reopen {num}']) and
            (row[f'Exit {num}'] != np.nan)  and
            (row[f'Reopen {num}'] != np.nan) and
            (row[f'Exit {num}'] > row['Initial IFSP Date']) and
            (row[f'Exit {num}'] < row['Exit Date']) and
            (row[f'Reopen {num}'] > row['Initial IFSP Date']) and
            (row[f'Reopen {num}'] < row['Exit Date'])):
            mult_diff_days = (row[f'Reopen {num}'] - row[f'Exit {num}']).days
            new_diff += mult_diff_days
            subset_expanded.at[index, 'diff_days_new'] = new_diff
        else:
            pass

In [ ]:
subset_expanded

In [ ]:
subset_expanded.loc[subset_expanded['Child ID'] == 433605]

In [ ]:
subset_expanded.loc[subset_expanded['diff_days_new'] < 0]

**Recalculated Days of Service**

In [ ]:
extract_time = subset_expanded[['Child ID', 'diff_days_new']]

In [ ]:
subset_multi_dates = pd.merge(TEIS_multiple_dates, extract_time, on = 'Child ID', how = 'left')
subset_multi_dates.head(2)

In [ ]:
subset_multi_dates['total_diff'] = subset_multi_dates['Days from I-IFSP to Exit'] - subset_multi_dates['diff_days_new']
subset_multi_dates.head(3)

## Explore the Reasons that Children Exit the Program

How often is it the case that children are aging out of the program compared to other reasons for leaving? 

In [ ]:
round(TEIS['Exit Reason'].value_counts(), 4)

What are the reasons that children are leaving and later coming back? 
    

In [ ]:
round(TEIS_reopened['Exit Reason'].value_counts(), 4)

In [ ]:
single_date_reasons = pd.concat([TEIS_single_date['Exit Reason'].value_counts(), 
                                 TEIS_single_date['Exit Reason'].value_counts(normalize = True).mul(100)],axis = 1, keys = ('Count', 'Percentage'))
round(single_date_reasons, 3)

In [ ]:
single_date_reasons.to_excel("single_date_charts.xlsx")

In [ ]:
multi_date_reasons = pd.concat([TEIS_multiple_dates['Exit Reason'].value_counts(), 
           TEIS_multiple_dates['Exit Reason'].value_counts(normalize = True).mul(100)],axis = 1, keys = ('Count', 'Percentage'))
round(multi_date_reasons, 3)

In [ ]:
multi_date_reasons.to_excel("reasons_chart.xlsx") 

In [ ]:
TEIS_reopened[['Exit Reason','Exit Note']]

In [ ]:
TEIS_multiple_dates[['Exit Reason', 'Exit Note']]

In [ ]:
TEIS_reopened.loc[TEIS_reopened['Exit Reason'] == '618 - Attempts to contact unsuccessful'].head(2)

In [ ]:
TEIS_reopened.loc[TEIS_reopened['Exit Reason'] == '618 - Part B eligibility not determined']

How often do children have less than six months of service for reasons other than aging out?

&emsp; Children with Multiple Reopen/Exit Dates

In [ ]:
subset_multi_dates.loc[subset_multi_dates['total_diff'] <= 180].shape

In [ ]:
subset_multi_

&emsp; Children with One Reopen/Exit Date

In [ ]:
TEIS_single_date.loc[TEIS_single_date['Real IFSP to Exit Time (Days)'] <= 180].shape